In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train = pd.read_csv('/kaggle/input/DontGetKicked/training.csv')
test = pd.read_csv('/kaggle/input/DontGetKicked/test.csv')
sub = pd.read_csv('/kaggle/input/DontGetKicked/example_entry.csv')

## 아래 ALLDATA 처리 부분을 보시면 됩니다
### 1. Label Encoder 를 Train + Test 로 처리 
### 2. CatBoost 사용 + cat_features 옵션 적용

In [ ]:
pd.options.display.max_columns = 999
train.head(3)

In [ ]:
train.dtypes

In [ ]:
train['Make'].unique()

In [ ]:
train['Auction'].unique()

In [ ]:
train['Auction'].value_counts()

In [ ]:
# train['Auction'] = train['Auction'].replace({'ADESA' :0, 'OTHER' :1, 
                                           #  'MANHEIM' :2})

In [ ]:
train.head(3)

In [ ]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

In [ ]:
# train['Make'] = le.fit_transform(train['Make'])

In [ ]:
train

In [ ]:
train['PurchDate'] = pd.to_datetime(train['PurchDate'])
test['PurchDate'] = pd.to_datetime(test['PurchDate'])

train['PurchYear'] = train['PurchDate'].dt.year
train['PurchMonth'] = train['PurchDate'].dt.month
# train

test['PurchYear'] = test['PurchDate'].dt.year
test['PurchMonth'] = test['PurchDate'].dt.month

In [ ]:
train2 = train.drop(columns = ['PurchDate', 'IsBadBuy'])
test2 = test.drop(columns = ['PurchDate'])

In [ ]:
c = train2.columns[train2.dtypes == object]
c

In [ ]:
# le = LabelEncoder()
# le_count = 0

# # Iterate through the columns
# for col in train2:
#     if train2[col].dtype == 'object':
        

#         # Train on the training data
#         le.fit(train2[col])
#         # Transform both training and testing data
#         train2[col] = le.transform(train2[col])
#         test2[col] = le.transform(test2[col])

#         # Keep track of how many columns were label encoded
#         le_count += 1

In [ ]:
# for i in c:
#    #  print(test2[i])
#     train2[i] = le.fit_transform(train2[i])

# d = test2.columns[test2.dtypes == object]
# for i in d:
#    #  print(test2[i])
#     test2[i] = le.fit_transform(test2[i])

In [ ]:
# c = train2.columns[train2.dtypes == object]
# d = train2.columns[train2.dtypes != object]

# train3 = train2.drop(columns = d)
# train4 = train2.drop(columns = c)

# train3 = pd.get_dummies(train3)

In [ ]:
# final_train = pd.concat([train3, train4], axis = 1)
# final_train

In [ ]:
# e = test2.columns[test2.dtypes == object]
# f = test2.columns[test2.dtypes != object]

# test3 = test2.drop(columns = d)
# test4 = test2.drop(columns = c)

# test3 = pd.get_dummies(test3)

# final_test = pd.concat([test3, test4], axis = 1)
# final_test

### ALLDATA 처리 --> train, test 데이터 불러온 뒤 이어서 바로 실행 (제출까지 한 셀에 작성)

In [ ]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

alldata = pd.concat([train, test])

alldata['PurchDate'] = pd.to_datetime(alldata['PurchDate'])

alldata['PurchYear'] = alldata['PurchDate'].dt.year
alldata['PurchMonth'] = alldata['PurchDate'].dt.month

alldata2 = alldata.drop(columns = ['PurchDate', 'IsBadBuy'])

c = alldata2.columns[alldata2.dtypes == object]

for i in c :
    print(i)
    alldata2[i] = le.fit_transform(list(alldata2[i]))

train2 = alldata2[:len(train)]
test2 = alldata2[len(train):]

from catboost import CatBoostClassifier

model = CatBoostClassifier(learning_rate = 0.1, verbose = 500, cat_features = ['Auction', 'Make', 'Model', 'Trim', 'SubModel', 'Color', 'Transmission',
       'WheelType', 'Nationality', 'Size', 'TopThreeAmericanName', 'PRIMEUNIT',
       'AUCGUART', 'VNST'])



model.fit(train2, train['IsBadBuy'])
result = model.predict_proba(test2)

sub['IsBadBuy'] = result[:,1]
sub.to_csv('base.csv', index = 0)

In [ ]:
c = train2.columns[train2.dtypes == object]
d = test2.columns[test2.dtypes == object]

for i in c :
    print(i)
    train2[i] = le.fit_transform(list(train2[i]))

for i in d :
    test2[i] = le.fit_transform(list(test2[i]))

In [ ]:
from catboost import CatBoostClassifier

model = CatBoostClassifier(learning_rate = 0.1, verbose = 500, cat_features = ['Auction', 'Make', 'Model', 'Trim', 'SubModel', 'Color', 'Transmission',
       'WheelType', 'Nationality', 'Size', 'TopThreeAmericanName', 'PRIMEUNIT',
       'AUCGUART', 'VNST'])

In [ ]:
# train2.dtypes

In [ ]:
# train2.isnull().sum()

In [ ]:
# train2 = train2.fillna(-1)
# test2 = test2.fillna(-1)

In [ ]:
# from sklearn.ensemble import RandomForestClassifier
# from xgboost import XGBClassifier

# model = XGBClassifier(learning_rate = 0.1)
# rf = RandomForestClassifier()

In [ ]:
model.fit(train2, train['IsBadBuy'])

In [ ]:
result = model.predict_proba(test2)

In [ ]:
result
# [   ,   ]

In [ ]:
sub['IsBadBuy'] = result[:,1]

In [ ]:
sub

In [ ]:
sub.to_csv('base.csv', index = 0)